In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision


print(torch.cuda.is_available())
if torch.cuda.is_available():
    print("Cuda is Availabe")
else:
    print("Cuda Can't be found")

# Just the device selection options
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

True
Cuda is Availabe
Using cuda device


## 

In [2]:
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.act1 = nn.Tanh()
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        self.act2 = nn.Tanh()
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0)
        self.act3 = nn.Tanh()

        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(1*1*120, 84)
        self.act4 = nn.Tanh()
        self.fc2 = nn.Linear(84, 10)
        
    def forward(self, x):
        # input 1x28x28, output 6x28x28
        x = self.act1(self.conv1(x))
        # input 6x28x28, output 6x14x14
        x = self.pool1(x)
        # input 6x14x14, output 16x10x10
        x = self.act2(self.conv2(x))
        # input 16x10x10, output 16x5x5
        x = self.pool2(x)
        # input 16x5x5, output 120x1x1
        x = self.act3(self.conv3(x))
        # input 120x1x1, output 84
        x = self.act4(self.fc1(self.flat(x)))
        # input 84, output 10
        x = self.fc2(x)
        return x

In [3]:
# Load MNIST data
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0,), (128,)),
])
train = torchvision.datasets.MNIST('data', train=True, download=True, transform=transform)
test = torchvision.datasets.MNIST('data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(train, shuffle=True, batch_size=100)
testloader = torch.utils.data.DataLoader(test, shuffle=True, batch_size=100)

import os

optimizer = None
loss_fn = None

def load_model():
    global model, optimizer, loss_fn
    print (os.path.curdir)
    model = LeNet5().to(device)
    optimizer = optim.Adam(model.parameters())
    loss_fn = nn.CrossEntropyLoss()
    
    MODEL_FILENAME='/home/vaisakh/developer/SysML/Project/PruningTest/model.sav'
    if (os.path.isfile(MODEL_FILENAME) != True):
        __train__(model, 10)
        torch.save(model.state_dict(),MODEL_FILENAME)
        print (f'Model saved to {MODEL_FILENAME}')
    else:
        model.load_state_dict(torch.load(MODEL_FILENAME))
        model.to(device)
        model.eval()

def __test__ (model__)-> float:
    model__.eval()
    acc = 0
    count = 0
    for X_batch, y_batch in testloader:
        X_batch = X_batch.to(device); y_batch = y_batch.to(device)
        y_pred = model__(X_batch)
        acc += (torch.argmax(y_pred, 1) == y_batch).float().sum()
        count += len(y_batch)
    acc = acc / count
    acc__ = acc.cpu()
    return acc__.numpy()*100
    

def __train__ (model__, n_epochs=10):
    for epoch in range(n_epochs):
        model__.train()
        for X_batch, y_batch in trainloader:
            X_batch = X_batch.to(device); y_batch = y_batch.to(device)
            y_pred = model__(X_batch)
            loss = loss_fn(y_pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print("Epoch %d: model accuracy %.2f%%" % (epoch, __test__(model__)))

In [4]:
import pandas as pd
import datetime, time,threading, psutil

class SystemStatsGatherer:
    def __init__(self,  interval=2):
        self.temp_dict = {}
        self.not_firstime = False
        self.diskio_fieldname = ['read_count', 'write_count','read_bytes', 'write_bytes', 'read_time', 'write_time', 'read_merged_count', 'write_merged_count', 'busy_time']
        self.diskio_fieldvals = [0.0 , 0.0, 0.0, 0.0 , 0.0, 0.0, 0.0 , 0.0, 0.0]
        self.swap_mem_fieldname = ['total', 'used', 'free', 'percent', 'sin', 'sout']
        self.swap_mem_fieldvals = [0.0 , 0.0, 0.0, 0.0 , 0.0, 0.0]
        self.virtual_mem_fieldname = ['total', 'available','percent', 'used', 'free', 'active', 'inactive', 'buffers', 'cached', 'shared', 'slab']
        self.virtual_mem_fieldvals = [0.0 , 0.0, 0.0, 0.0 , 0.0, 0.0, 0.0 , 0.0, 0.0, 0.0, 0.0]
        self.tempsensor_fieldname = ['current','high', 'critical']
        self.tempsensor_fieldvals = [0.0 , 0.0, 0.0]
        self.samplerThread = threading.Thread(target=self.samplerThread)
        self.samplingInterval = interval
        self.stopThread = False


    def SampleTemp(self):

        tempsensor_name = ""
        res_temp = psutil.sensors_temperatures()
        res_diskio = psutil.disk_io_counters(perdisk=False, nowrap=True)
        res_swapmem = psutil.swap_memory()
        res_virtmem = psutil.virtual_memory()
        ctr = 0
        global not_firstime
        ts = datetime.datetime.now()
        if (self.not_firstime == False):
            self.temp_dict['time'] = [str(ts)]
        else:
            self.temp_dict['time'] += [str(ts)]

        ## get CPU Load averages
        loadavgs = [x / psutil.cpu_count() * 100 for x in psutil.getloadavg()]
        if (self.not_firstime == False):
            self.temp_dict['cpuloadavg_1min'] = loadavgs[0]
            self.temp_dict['cpuloadavg_5min'] = loadavgs[1]
            self.temp_dict['cpuloadavg_15min'] = loadavgs[2]
        else:
            self.temp_dict['cpuloadavg_1min'] += loadavgs[0]
            self.temp_dict['cpuloadavg_5min'] += loadavgs[1]
            self.temp_dict['cpuloadavg_15min'] += loadavgs[2]


        ## Process the Virtual Memory readings
        ctr = 0
        for val in res_virtmem:
            if (self.not_firstime == False):
                self.temp_dict['vm_'+self.virtual_mem_fieldname[ctr]] = [val]
            else:
                self.temp_dict['vm_'+self.virtual_mem_fieldname[ctr]] += [val]
            ctr += 1
        ## Process the Swap Memory readings
        ctr = 0
        for val in res_swapmem:
            if (self.not_firstime == False):
                self.temp_dict['swap_'+self.swap_mem_fieldname[ctr]] = [val]
            else:
                self.temp_dict['swap_'+self.swap_mem_fieldname[ctr]] += [val]
            ctr += 1
        ## Process the DiskIO readings
        ctr = 0
        for val in res_diskio:
            if (self.not_firstime == False):
                self.temp_dict['diskio_'+self.diskio_fieldname[ctr]] = [val]
            else:
                self.temp_dict['diskio_'+self.diskio_fieldname[ctr]] += [val]
            ctr += 1
        ## Process the Temperature readings
        for key,values in res_temp.items():
            # print (key,' => ',values)
            for temp_elems in values:
                # print (temp_elems)
                tempsensor_name = key
                ctr = 0;title_field = True
                for val in temp_elems:
                    if (title_field == True):
                        tempsensor_name +=  '-'+str(val)
                        title_field = False
                    else:
                        self.tempsensor_fieldvals[ctr] = val
                        ctr += 1
                        
                    # print(key,'-', '=>',tempsensor_fieldname[ctr],'-->',sensor_val)
                # print (tempsensor_name,'=>',tempsensor_fieldvals)
                ctr = 0
                for elems in self.tempsensor_fieldvals:
                    # print (tempsensor_name+'-'+tempsensor_fieldname[ctr],'=>',elems)
                    if (self.not_firstime == False):
                        self.temp_dict[tempsensor_name+'-'+self.tempsensor_fieldname[ctr]] = [elems]
                    else:
                        self.temp_dict[tempsensor_name+'-'+self.tempsensor_fieldname[ctr]] += [elems]
                    ctr += 1
        self.not_firstime = True

    def getReadings(self):
        return self.temp_dict
    
    def samplerThread(self):
        while (self.stopThread != True):
            self.SampleTemp()
            time.sleep(self.samplingInterval)
        return
    
    def startSampling(self):
        self.stopThread = False
        self.samplerThread.start()
        return 

    def stopSampling(self):
        self.stopThread = True
        self.samplerThread.join()
        return 


In [5]:
import torch.nn.utils.prune as prune
import numpy as np
import copy

import pandas as pd
import io
from torch.profiler import profile, record_function, ProfilerActivity


# COLUMN_NAMES = ['prune-config', 'prune-ratio','vm-percent','swap-percent','cpuloadavg-1min', 'cpuloadavg-5min', 'ratio','memory_consumption','acc']
df_final = pd.DataFrame()
load_model()
for ratio in np.arange(0.0,1.0,0.1):    
    statsMon = SystemStatsGatherer()
    statsMon.startSampling()
    ratio = round(ratio,2)
    print (ratio )
    parameters = ((model.conv1, "weight"),(model.conv2, "weight"),(model.fc1, "weight"),(model.fc2, "weight"),)
    prune.global_unstructured(parameters,pruning_method=prune.L1Unstructured,amount=ratio,)
    for items in parameters:
        # print(items[0],'=',items[1])
        prune.remove(items[0],items[1])

    prof = torch.profiler.profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA, torch.profiler.ProfilerActivity],profile_memory=True, record_shapes=True,with_flops=True)
    print ('Profiling run completed.\nCompiling result table')
    prof.start()
    acc = __test__(model)
    prof.stop()

    print ('Profiling run completed.\nCompiling result table')
    prof_result = prof.key_averages().table(sort_by='cpu_memory_usage', row_limit=100)
    prof_result_back = prof_result

    #Trim off the header and footer of the results
    prof_result_back = prof_result_back.rsplit("\n",3)[0]
    prof_result__ = ""
    for line in prof_result_back:
        if "-" not in line.split():
            prof_result__ += line 
    prof_result_back = prof_result__
    # print (prof_result_back)
    df = pd.read_csv(io.StringIO(prof_result_back), sep="\s\s+")

    # pd.set_option('display.max_rows', 500)
    # display(df)

    memory_consumption_str = df[(df['Name']=='[memory]')]['CPU Mem'].iloc[0]
    flops = df[(df['Name'] == 'aten::conv2d')]['Total FLOPs'].iloc[0]

    memory_consumption = 0
    if ('Gb' in memory_consumption_str):
        memory_consumption =  float(memory_consumption_str.split()[0])*1024*1024*1024
    elif ('Mb' in memory_consumption_str):
        memory_consumption =  float(memory_consumption_str.split()[0])*1024*1024
    elif ('Kb' in memory_consumption_str):
        memory_consumption =  float(memory_consumption_str.split()[0])*1024
    elif ('b' in memory_consumption_str):
        memory_consumption =  float(memory_consumption_str.split()[0])

    print(ratio, memory_consumption, flops, acc)
    statsMon.stopSampling()
    print('Collecting System Stats')
    df_stats = pd.DataFrame.from_dict(statsMon.getReadings())
    agg_df = df_stats[['vm_percent','swap_percent','cpuloadavg_1min', 'cpuloadavg_5min']]
    agg_df__ =  agg_df.mean()
    agg_df__['ratio'] = ratio
    agg_df__['memory_consumption'] = memory_consumption
    agg_df__['acc'] = acc
    agg_df__['prune_config'] = 'global_unstructured-L1Unstructured'
    agg_df__['prune_ratio'] = ratio
    agg_df__['flops'] = flops
    
    display(agg_df__)
    df_final = pd.concat([df_final,agg_df__], axis=1)
    print('Aggregated stats')
    display(df_final)
    del prof_result
del model, loss_fn, optimizer



.
0.0
Profiling run completed.
Compiling result table


STAGE:2024-03-31 05:39:07 864071:864071 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-03-31 05:39:26 864071:864071 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-03-31 05:39:27 864071:864071 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


Profiling run completed.
Compiling result table


/tmp/ipykernel_864071/2108505725.py:39: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(io.StringIO(prof_result_back), sep="\s\s+")


0.0 188638822.4 48672000000.0 98.17166924476624


...

/tmp/ipykernel_864071/2108505725.py:39: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(io.StringIO(prof_result_back), sep="\s\s+")


0.9 188638822.4 48672000000.0 10.441666841506958


vm_percent                                     86.316071
swap_percent                                         0.2
cpuloadavg_1min                               573.458862
cpuloadavg_5min                               612.902832
ratio                                                0.9
memory_consumption                           188638822.4
acc                                            10.441667
prune_config          global_unstructured-L1Unstructured
prune_ratio                                          0.9
flops                                      48672000000.0
dtype: object

Aggregated stats


,0,0,0,0,0,0,0,0,0,0
vm_percent,50.548214,77.749091,82.203571,83.975439,83.724528,85.278182,84.633929,85.963636,85.607547,86.316071
swap_percent,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
cpuloadavg_1min,603.95813,917.712402,853.417969,615.133667,620.040894,503.22876,785.812378,676.71814,546.044922,573.458862
cpuloadavg_5min,577.865601,784.161377,804.391479,755.368042,682.247925,582.766724,660.687256,700.219727,603.436279,612.902832
ratio,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
memory_consumption,188638822.4,188638822.4,188638822.4,188638822.4,188638822.4,188638822.4,188638822.4,188638822.4,188638822.4,188638822.4
acc,98.171669,98.155004,98.10667,97.643334,96.025002,84.548336,31.548333,13.753334,10.441667,10.441667
prune_config,global_unstructured-L1Unstructured,global_unstructured-L1Unstructured,global_unstructured-L1Unstructured,global_unstructured-L1Unstructured,global_unstructured-L1Unstructured,global_unstructured-L1Unstructured,global_unstructured-L1Unstructured,global_unstructured-L1Unstructured,global_unstructured-L1Unstructured,global_unstructured-L1Unstructured
prune_ratio,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
flops,48672000000.0,48672000000.0,48672000000.0,48672000000.0,48672000000.0,48672000000.0,48672000000.0,48672000000.0,48672000000.0,48672000000.0


In [6]:
DF_SAVEFILENAME = '/home/vaisakh/developer/SysML/Project/PruningTest/LeNet5-experiment.csv'
df_final_transpose = df_final.transpose().reset_index(drop=True)
display(df_final_transpose)
df_final_transpose.to_csv(DF_SAVEFILENAME, encoding='utf-8', sep=',')

,vm_percent,swap_percent,cpuloadavg_1min,cpuloadavg_5min,ratio,memory_consumption,acc,prune_config,prune_ratio,flops
0,50.548214,0.2,603.95813,577.865601,0.0,188638822.4,98.171669,global_unstructured-L1Unstructured,0.0,48672000000.0
1,77.749091,0.2,917.712402,784.161377,0.1,188638822.4,98.155004,global_unstructured-L1Unstructured,0.1,48672000000.0
2,82.203571,0.2,853.417969,804.391479,0.2,188638822.4,98.10667,global_unstructured-L1Unstructured,0.2,48672000000.0
3,83.975439,0.2,615.133667,755.368042,0.3,188638822.4,97.643334,global_unstructured-L1Unstructured,0.3,48672000000.0
4,83.724528,0.2,620.040894,682.247925,0.4,188638822.4,96.025002,global_unstructured-L1Unstructured,0.4,48672000000.0
5,85.278182,0.2,503.22876,582.766724,0.5,188638822.4,84.548336,global_unstructured-L1Unstructured,0.5,48672000000.0
6,84.633929,0.2,785.812378,660.687256,0.6,188638822.4,31.548333,global_unstructured-L1Unstructured,0.6,48672000000.0
7,85.963636,0.2,676.71814,700.219727,0.7,188638822.4,13.753334,global_unstructured-L1Unstructured,0.7,48672000000.0
8,85.607547,0.2,546.044922,603.436279,0.8,188638822.4,10.441667,global_unstructured-L1Unstructured,0.8,48672000000.0
9,86.316071,0.2,573.458862,612.902832,0.9,188638822.4,10.441667,global_unstructured-L1Unstructured,0.9,48672000000.0
